# SPGD: Search Party Gradient Descent algorithm, a Simple Gradient-Based Parallel Algorithm for Bound-Constrained Optimization

Link: https://www.mdpi.com/2227-7390/10/5/800

#### This is a preliminary code written for the SPGD paper.
The code is not efficient, and has lots of room for improvement.
We plan to release an updated version of this code in future.

In [1]:
import numpy as np
import random
import threading
import time
import math, statistics

### Benchmark Functions

In [39]:
def f(x): #F1Ackley 
    x,y = x[0],x[1]
    '''
    Ackley Function
    wikipedia: https://en.wikipedia.org/wiki/Ackley_function
    global minium at f(x=0, y=0) = 0
    bounds: -35<=x,y<=35
    '''
    return (-20 * np.exp(-0.02 * np.sqrt(0.5 * (x*x + y*y))) -
            np.exp(0.5 * (np.cos(2.0*np.pi*x) + np.cos(2*np.pi*y))) + np.e + 20) #Ackley
mrnge = [-35,35]
optimum = 0
dim = 2

In [15]:
def f(x): #F2beale
    x,y = x[0],x[1]
    '''
    Beale Function
    global minimum: f(x=3, y=0.5) = 0
    bounds: -4.5 <= x, y <= 4.5
    '''
    return ((1.500 - x + x*y)**2 +
            (2.250 - x + x*y**2)**2 +
            (2.625 - x + x*y**3)**2)  #Beale
mrnge = [-5,5]
optimum = 0
dim = 2

In [7]:
def f(x): #F3Egg Holder
    x,y = x[0],x[1]
    '''
    Eggholder Function
    global minimum: f(x=512, y=404.2319) = -959.6407
    bounds: -512 <= x, y <= 512
    '''

    return (-(y+47)*np.sin(np.sqrt(abs((x/2.0) + (y+47)))) -
            x*np.sin(np.sqrt(abs(x-(y+47)))))
mrnge = [-512,512]
optimum = -959.6407
dim = 2

In [112]:
def f(xy):#F4Gold-Stein
    '''
    Goldstein-Price Function
    global minimum: f(0, -1) = 3
    bounds: -2 <= x, y <= 2
    '''
    x, y = xy[0], xy[1]
    return ((1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) *
            (30 + (2*x-3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2)))
mrnge = [-2,2]
optimum = 3
dim = 2

In [20]:
def f(x):#F5Matyas
    x,y = x[0],x[1]
    '''
    Matyas Function
    global minimum: f(x=0, y=0) = 0
    bounds: -10 <= x, y <= 10
    '''
    return 0.26*(x**2 + y**2) - 0.48*x*y #matyas
mrnge = [-10,10]
optimum = 0
dim = 2

In [43]:
def f(x): #F6Sahaffer
    x,y = x[0],x[1]
    '''
    Schaffer Function N.4
    Reference: https://www.sfu.ca/~ssurjano/schaffer4.html
    global minimum:
        f(x=0, y= 1.25313) = 0.292579
        f(x=0, y=-1.25313) = 0.292579
    bounds: -100 <= x, y <= 100
    '''
    return 0.5 + (np.cos(np.sin(x**2 - y**2))**2 - 0.5)/(1+0.001*(x**2 + y**2))**2 #N.4
mrnge = [-100,100]
optimum = 0.292579
dim = 2

In [65]:
def f(xy): #Colville
    '''Colville Function
    Parameters
    ----------
        xy : list
    Returns
    -------
        float
    Notes
    -----
    Bounds: The Colville function is a 4-dimensional function usually evaluated
    on the hypercube defined by x_i in [-10, 10] for i=1,2,3,4.
    Global minimum: f(x)=0 at x=[1,1,1,1]
    References
    ----------
    A.-R. Hedar, “Global Optimization Test Problems”
    '''

    x1, x2, x3, x4 = xy[0], xy[1], xy[2], xy[3]
    a = 100*(x1**2 - x2)**2
    b = (x1-1)**2
    c = (x3-1)**2
    d = 90*(x3**2 - x4)**2
    e = 10.1*((x2-1)**2 + (x4-1)**2)
    f = 19.8*(x2-1)*(x4-1)
    return a + b + c + d + e + f
mrnge = [-10,10]
optimum = 0
dim = 4

In [10]:
def f(xy): #griewank
    '''Griwank Function
    Bounds: x_i in [-600, 600] for all i=1,...,d
    Global minimum: f(x)=0 at x=(0,...,0)

    '''
    a, b, = 0, 1
    for i, v in enumerate(xy):
        a += v**2 / 4000.0
        b *= np.cos(v/np.sqrt(i+1))
    return a - b + 1
mrnge = [-600,600]
optimum = 0
dim = 5

In [93]:
def f(x, m=10): #Michaelwicz
    '''Michalewicz Function
    Notes
    -----
    The parameter m defines the steepness of they valleys and ridges; a larger m leads to a more difficult search. The recommended value of m is m = 10.
    global minimum for 2D: f(x)=-1.8013 at x*=(2.20,1.57)
    bounds: x_i in [0, π] for i=1,..., d
    '''
    d = len(x)
    result = 0
    for i in range(d):
        result -= np.sin(x[i])*(np.sin((i+1)*x[i]**2/np.pi))**(2*m)
    return result
mrnge = [0,4]
optimum = -4.687658
dim = 5

In [81]:
def f(x): #Rosenbrock
    '''
    Rosenbrock Function
    wikipedia: https://en.wikipedia.org/wiki/Rosenbrock_function
    global minimum:
        n=2 -> f(1,1)=0
    bounds:
        -5 <= x_i <= +10
        1 <= i <= n
    '''
    total = 0
    for i in range(len(x)-1):
        total += 100*(x[i+1] - x[i]*x[i])**2 + (1-x[i])**2
    return total
mrnge = [-30,30]
optimum = 0
dim = 10

In [95]:
def f(x): #10D_zakharov
    '''Zakharov Function
    '''
    a, b = 0, 0
    for i, val in enumerate(x):
        a += val**2
        b += 0.5*i*val
    return a + b**2 + b**4
mrnge = [-5,10]
optimum = 0
dim = 10

In [86]:
def f(xy): #Rotated Ellipsoid
    '''Rotated Hyper-Ellipsoid
    Bounds: the rotated hyper-ellipsoid is usually evaluated on the hypercube
    defined by x_i in [-65.536, 65.536] for all i=1,...,d
    Global minimum: f(x)=0 at x=(0,...,0)
    References
    ----------
    Molga, M., & Smutnicki, C. Test functions for optimization needs (2005)
    '''
    a = 0
    for i in range(0, len(xy)): a += sum([xy[j]**2 for j in range(0, i)])
    return a
mrnge = [-66,66]
optimum = 0
dim = 10

In [8]:
def f(x): #Rastrigin
    '''Rastrigin Function
    Notes
    -----
    Bounds: -5.12 <= x_i <= 5.12 for all i=1,...,d
    Global minimum: f(x)=0 at x=(0,...,0)
    References
    ----------
    wikipedia: https://en.wikipedia.org/wiki/Rastrigin_function
    '''
    return len(x)*10.0 +  sum([item*item - 10.0*np.cos(2.0*np.pi*item) for item in x]) #Rastrigin
mrnge = [-6,6]
optimum = 0
dim = 20

In [23]:
def f(xy): #Holder Table
    '''
    Holder Table Function
    global minimum:
        f(x= 8.05502, y= 9.66459) = -19.2085
        f(x=-8.05502, y= 9.66459) = -19.2085
        f(x= 8.05502, y=-9.66459) = -19.2085
        f(x=-8.05502, y=-9.66459) = -19.2085
    bounds: -10 <= x, y <= 10
    '''
    x, y = xy[0], xy[1]
    return -abs(np.sin(x)*np.cos(y)*np.exp(abs(1-(np.sqrt(x**2 + y**2)/np.pi))))
mrnge = [-10,10]
optimum = -19.2085

In [21]:
def f(xy): #tripod
    '''Tripod Function
    Parameters
    ----------
        xy : list
    Returns
    -------
        float
    Notes
    -----
    Bounds: x_i in [-100, 100] for i=1,2
    Global minimum: f(x)=0 at x=[0,-50]
    References
    ----------
    S. Rahnamyan, H. R. Tizhoosh, N. M. M. Salama, “A Novel Population
    Initialization Method for Accelerating Evolutionary Algorithms”
    Computers and Mathematics with Applications, vol. 53, no. 10,
    pp. 1605-1614, 2007.
    '''

    x1, x2 = xy[0], xy[1]
    p_x1 = 1 if x1>=0 else 0
    p_x2 = 1 if x2>=0 else 0

    a = p_x2 * (1 + p_x1)
    b = abs(x1 + 50*p_x2*(1-2*p_x1))
    c = abs(x2 + 50*(1-2*p_x2))
    return a + b + c
mrnge = [-100,100]
optimum = 0

## SPGD Begin

In [7]:
def derivative (arr, pos):
    h = 0.000000001
    temp = arr.copy()
    temp[pos] = temp[pos] + h
    num = f(temp) - f(arr)
    return num/h

In [8]:
def printing (minimalist):
    lst=[]
    for i in range (len(minimalist)):
        lst.append(f(minimalist[i]))
    minima_arr.append(min (lst))

In [9]:
def descent(tid, w, a, e):
    for i in range(e*Iter_P_Ep, e * Iter_P_Ep + Iter_P_Ep):
        for pos in range(dim):
            if (w[pos] < rnge[0] or w[pos] > rnge[1]):
                w[pos] = random.randint(rnge[0],rnge[1])
                
        V[tid][i] = w
        for pos in range(dim):
            w[pos] = w[pos] - a * derivative(w,pos)
            
             
        global CLM
        if f( V[tid][i]) < fval[0]:
            fval[0] = f( V[tid][i])
            CLM = V[tid][i]

In [26]:
def check_pre_convrg ():
    pre_convg = []       
    global Turn, skip
    for i in range(ep-5, len(minimalist)):
        pre_convg.append(f(minimalist[i]))

    if(statistics.stdev(pre_convg) == 0 and Turn == 0):
        lrn = np.linspace(0.1, 0.001, num=Tc)
        Turn +=1
        skip = 3

    if(skip > 0):
        skip = skip -1

    elif(statistics.stdev(pre_convg) == 0 and Turn == 1 and skip ==0):
        lrn = np.linspace(0.01, 0.0001, num=Tc)
        Turn +=1
        skip = 3

    elif(statistics.stdev(pre_convg) == 0 and Turn == 2 and skip == 0):
        lrn = np.linspace(0.001,0.000001, num = Tc)
        Turn +=1 

In [30]:
def reduce_search_circle ():
    if(ep % 5 == 0 and ep !=0):
        dummy = 5
        tempmin, tempmax=[],[]
        for i in range(ep-dummy, len(minimalist)):
            tempmin.append(math.floor(min(minimalist[i])))
            tempmax.append(math.ceil(max(minimalist[i])))
        rnge[0]= min(tempmin)
        rnge[1] = max(tempmax)
        if (rnge[0] == rnge[1]):
            rnge[0],rnge[1] = mrnge[0],mrnge[1]

In [66]:
minima_arr = []
avg_epp = []
start = time.perf_counter()
for run in range(50):    #Running SPGD 50 times for statistcal analysis
    print("Run: ", run)
    
    #Begin Intialization of various variables
    rnge = [mrnge[0],mrnge[1]]
    fval, CLM = [9999999],np.zeros(dim)
    Turn, skip = 0,-1
    Episodes, Iter_P_Ep = 50, 20
    LocMinima,minimalist, epsilon, Tc =[] ,[], 1, 25 #Threadcount
    decay = epsilon/Episodes

    V,T = np.zeros((Tc, Episodes * Iter_P_Ep, dim)), [0] * Tc
    lrn = np.linspace(1, 0.001, num=Tc)
    init_xy = np.random.uniform(rnge[0],rnge[1], size=(Tc, dim))
    
    #End Intialization
    
    #Begin SPGD Execution
    for ep in range(Episodes):
        
        for i in range(Tc):
            T[i] = threading.Thread(target=descent, args=(i, init_xy[i],lrn[i], ep))    
            T[i].start()  
        for i in range(Tc):
            T[i].join()

        Edge_Loc = ep*Iter_P_Ep + (Iter_P_Ep-1)

        minimalist.append(CLM) #Remember Local Minima Encountered
        
        reduce_search_circle()        
        if(ep > 5):
            check_pre_convrg()
            
        #Exploitation Vs Exploration
        if(random.uniform(0,1) >= epsilon): #Exploitation
            for Tid in range(Tc):
                if (random.uniform(0,1) >= 0.2):
                    init_xy[Tid] = CLM #Assemble at best known location
                else: #Chooses to explore 20% of the time, to avoid loc minima.
                    for pos in range(dim):
                        init_xy[Tid][pos] = np.random.triangular(rnge[0],CLM[pos],rnge[1])

        else:
            #EXPLORATION
            for Tid in range (Tc):
                #Continue as and where you are
                if (random.uniform(0,1) >= 0.5): #Dont Change
                    init_xy[Tid] = V[Tid][Edge_Loc]
                else: #Pure Exploration
                    for pos in range(dim):
                        init_xy[Tid][pos] = np.random.triangular(rnge[0],CLM[pos],rnge[1])
                        
                        
        if(ep >10 ):
            convergence = []        
            for i in range(ep-10, len(minimalist)):
                convergence.append(f(minimalist[i]))
            if(statistics.stdev(convergence) == 0):
                print(run, ep, f(CLM))
                avg_epp.append(ep)
                break
        epsilon = epsilon - decay
    printing(minimalist)
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

Run:  0
0 20 0.0
Run:  1
1 16 0.0
Finished in 0.67 second(s)


In [35]:
#Summary
import statistics
print(min(minima_arr), max(minima_arr),statistics.mean(minima_arr) ,statistics.stdev(minima_arr))
j=0
for i in range(len(minima_arr)):
    if abs(minima_arr[i] - optimum )< 0.000001 :
        #print(j,"success", minima_arr[i])
        j+=1
print(j/50)
dum = avg_epp
for _ in range(50 - len(avg_epp)):
    dum.append(50)
statistics.mean(dum) 

9.063286299237916e-08 2.0419683007588674e-05 1.0255157935290526e-05 1.4374809212325405e-05
0.02


49.44